# Practical 3 - Validation
****
In this practical session you will explore how to validate the prediction models developed in practical 1.

As shown earlier, we can use the 'predict()' function in R to estimate predicted risks from our newly developed CPM.

We will then outline the process of how to use such predicted risks to validate the prediction model. As discussed in the lecture notes, this should be undertaken in new (independent) data. However, we do not have external data to validate our model. For purposes of illustration of the R code, we here test performance within the development dataset. We will call this the 'apparent performance' of the model. Both the calibration and discrimination of the model will be calculated.

It is often the case that we do not have external data, but dont worry there is no need to panic or split our data. We can perform internal validation. The latter part of this practical will illustrate how to perform bootstrap internal validation.

We will focus only on the model developed in practical 1. The majority of the validation techniques and code still hold for the survival models by evaluating at one or more timepoints. Where there are differences between the logistic and survival valdation, they are discused in the text. 


### Lets begin

First lets load the packages we will be using in the practical

In [ ]:
library(pROC)
library(DescTools)
library(Hmisc)
library(plyr)
library(boot)
library(MASS)
library(rms)

## Part 1

Begin by loading the data and subset, as we have done in previous practicals.

In [ ]:
smart <- readRDS("SMARTs_P2-3.rds")
smart <- subset(smart, select = c(outcome, TEVENT, EVENT, SEX, AGE, SBP, alcohol, CHOLO, BMIO, DIABETES, CARDIAC, SMOKING, AAA))
smart <- na.omit(smart)
attach(smart)
class(smart)
sapply(smart,class)

## Part 2

Fit the multivariable models selected from practical 1

In [ ]:
age3_spline <- rcs(AGE,3)

# Fit model selected from practicals 1 and 2
mod_log <- glm(outcome~SEX+SBP+DIABETES+age3_spline,family="binomial",x=TRUE,y=TRUE)
mod_log


## Part 3 apparent discrimination

Discrimination refers to the ability of a CPM to separate patients who will develop/experiance an outcome from those who will not.  
The C-statistic (concordance) is typically used to assess model discrimination. This is calculated as the concordant pairs of individuals. If we take two individuals, one who experienced the outcome and one who did not, and the individual who had the event was given the higher probability, then this pair of individuals are concordant.
We can also plot the receiver operator characteristic (ROC) curve. *Note - The area under this curve is equivalent to the C-statistic.*

### Run the below code for discrimination. Can you interpret the results?

In [ ]:
## Calculate apparent discrimination performance

In [ ]:
# Obtain the predicted probabilities for each patient
pred_prob <- predict(mod_log,type="response")

# Obtain the c statistic / AUC
c1 <- roc(outcome~pred_prob,ci=TRUE)
c1

# Plot the ROC curve
plot(roc(outcome,pred_prob))



The C-statistic can also be calculated for survival models but does not include censored individuals or individuals with the same event time. The D-statistic is often calculated instead. This was proposed by Royston and Sauerbrei and their paper can be found [here](https://onlinelibrary.wiley.com/doi/abs/10.1002/sim.1621).


## Part 4 apparent calibration

Lets begin by calculating calibration-in-the-large (CITL). This compares the mean of all predicted risks with the mean observed risk. This is often used to indicate the extent that predictions are systematically too low or too high. 
If CITL > 0, observed proportions are higher than the predicted probabilities.  
If CITL < 0, predicted probabilities are higher than the observed proportions.

We can also use the expected/observed ratio (E/O) by dividing the expected by the observed number of events (or probabilities).  
E/O is related to CITL and when E/O<1, CITL>0. Also, when E/O>1, CITL<0  
*Note - When overall prevelence is low, E/O as a ratio can look very large even when differences are smalll.*

In [ ]:
# Obtain the linear predictor for each patient
pred_LP <- predict(mod_log,type="link")

# CITL
mod_log_1 <- glm(outcome~offset(pred_LP),family="binomial")
coefficients(mod_log_1)
confint(mod_log_1)

# E/O
prop.table(table(outcome))
O <- prop.table(table(outcome))[2]
E <- mean(pred_prob)
E/O



### What does this show? Is it what you expected?why?


E/O for survival models use expected and observed event probabilities rather than the number of events. This can be calculated to one or more time points. Thereofre the expected/observed ratio at 2 years (24 months) is,  
$$E/O(t) = \frac{1-S_{exp}(t)}{1-S_{obs}(t)}$$



An alternative  way of testing calibraiton is by estimating the so-called calibration slope. This is obtained by fitting a logistic regression model to the observed outcomes and using the linear predictor of the model as the only covariate. A  calibration slope < 1 indicates the model is overfitted (too extreme risk predictions). NOTE - calibration intercept is the same as calibration in the large


### Here is the calibration slope. Can you interpret the result and is it what you expect?

In [ ]:
mod_log_2 <- glm(outcome~pred_LP,family="binomial",x=TRUE,y=TRUE)
coefficients(mod_log_2)
confint(mod_log_2)



For survival models you can obtain the the calibration slope by fitting a cox model to the LP as the only predictor.  
For example, *coxph(surv(TEVENT, EVENT)~pred_LP,x=TRUE,y=TRUE)*

Another way to assess calibration of a CPM is a so-called calibration plot, which we make by dividing the data into several (usually ten) equally sized groups based on the ordered probabilities, and calculating the observed and expected number of events in each group. One way of doing this is using the code below:

In [ ]:
# Visual assessment of calibration by risk groups

# create 10 risk groups
groups <- cut(pred_prob,breaks=quantile(pred_prob, prob = c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0)),labels=c(1:10),include.lowest=TRUE)
 
# average the observed and expected probabilities of patients in each risk group 
gpdata <- cbind(smart,groups,pred_prob)
obs <- (ddply(gpdata,~groups,summarise,mean=mean(as.numeric(outcome)))[,2])
exp <- ddply(gpdata,~groups,summarise,mean=mean(pred_prob))
obsn <- table(outcome,groups)[1,] 

# CIs for scatter points
lci = pmax(0,(obs - (1.96*(((obs*(1-obs))/obsn)^.5))))
uci = pmin(1,(obs + (1.96*(((obs*(1-obs))/obsn)^.5))))

# graph a simple calibration plot over 10 risk groups
par(pty="s")
plot(obs~exp[,2],xlim=c(0,1),ylim=c(0,1),col="red",ylab="Observed",xlab="Expected")
lines(c(0,1),c(0,1),lty=2)
for(i in 1:10){
lines(c(exp[i,2],exp[i,2]),c(lci[i],uci[i]),col="green")
}
h <- hist(pred_prob, breaks=50, plot=FALSE)
for(i in 1:length(h$mids)){
  lines(c(h$mids[i],h$mids[i]),c(rep(1,length(h$mids))[i],1-((h$counts[i]/max(h$counts))/10)))
}

# Add a loess smoother to the plot
obs_all <- predict(loess((as.numeric(outcome))~pred_prob,span=1))
lines_data <- data.frame(pred_prob,obs_all)
lines_data2 <- lines_data[order(pred_prob),] 
lines(lines_data2[,1],lines_data2[,2],col="blue")
legend(0.0,0.9,c("Risk groups","Reference line","95% CI","Loess"),col=c("red","black","green","blue"),lty=c(0,2,1,1),pch=c(1,NA,NA,NA),bty="n")


In [ ]:
par(pty="s")
plot(obs~exp[,2],xlim=c(0,.3),ylim=c(0,.3),col="red",ylab="Observed",xlab="Expected")
lines(c(0,1),c(0,1),lty=2)
for(i in 1:10){
lines(c(exp[i,2],exp[i,2]),c(lci[i],uci[i]),col="green")
}
h <- hist(pred_prob, breaks=50, plot=FALSE)
for(i in 1:length(h$mids)){
  lines(c(h$mids[i],h$mids[i]),c(rep(1,length(h$mids))[i],1-((h$counts[i]/max(h$counts))/10)))
}

# Add a loess smoother to the plot
obs_all <- predict(loess((as.numeric(outcome))~pred_prob,span=1))
lines_data <- data.frame(pred_prob,obs_all)
lines_data2 <- lines_data[order(pred_prob),] 
lines(lines_data2[,1],lines_data2[,2],col="blue")
legend(0.0,0.25,c("Risk groups","Reference line","95% CI","Loess"),col=c("red","black","green","blue"),lty=c(0,2,1,1),pch=c(1,NA,NA,NA),bty="n")


### What does this show

In [ ]:
#Brier score
BrierScore(mod_log)

The Brier score is also a measure of performance and ranges from 0 to 0.25, with lower values representing better performance. The brier score of 0.051 indicates good overall performance.

### Summary of above results
These results show that the CPM is well calibrated and has moderate discrimination. However, these performance estimates are misleading because they are based on the data for which the CPM was derived and are therefore subject to in-sample optimism.  

This optimism can be adjusted for by bootstrapped validation. This is the prefered method for internal validation of a prediction model.

## Part 5 bootstrap validation

In [ ]:

# You can obtain c-statistic and c-slope directly from Harrell's "validate" programme within 'rms'.
mod1 <- lrm(outcome~SEX+SBP+DIABETES+age3_spline,x=TRUE,y=TRUE)
set.seed(231398)
boot_1 <- validate(mod1,method="boot",B=50)
boot_1

# Note that this gives Dxy rather than c, however Dxy = 2*(c-0.5), i.e. c=(Dxy/2)+0.5
(boot_1[1,1]+1)/2
(boot_1[1,5]+1)/2


In [ ]:
## To get bootstrapped answers for CITL, c-statistic & c-slope it's a more manual approach:
manual_boot <- function(data,samples){
  results <- matrix(nrow = samples,ncol = 6)
  set.seed(231398)
  for (i in 1:samples) {
    samp_index <- sample(1:nrow(data), nrow(data), rep=TRUE) # create a sampling index vector
   
    bs_samp <- data[samp_index,] # index the orignal dataset using the sampling vector to give the bs sample
    model <- glm(outcome~SEX+age3_spline + DIABETES,family=binomial, data=bs_samp)  # Fit model to the bootstrap sample
    pr_bs <- predict(model,type="response") # predict probabilities from the bootstrap model in the bs sample
    lp_bs <- predict(model) # predict lp from the bootstrap model in the bs sample
    
    pr_test <- predict(model,type="response",newdata = data) # predict probabilities from the bootstrap model in the original sample
    lp_test <- predict(model, newdata = data) # predict lp from the bootstrap model in the original sample
    
    # calculate the apparent performance of the bootstrap model in the bs sample
    app_cstat_model <- roc(outcome~pr_bs,data=bs_samp)
    results[i,1] <- app_cstat_model$auc
    app_citl_model <- glm(outcome ~ offset(lp_bs),family=binomial, data=bs_samp)
    results[i,2] <- summary(app_citl_model)$coefficients[1,1]
    app_cslope_model <- glm(outcome ~ lp_bs,family=binomial(link='logit'), data=bs_samp)
    results[i,3] <- summary(app_cslope_model)$coefficients[2,1]
    
    # calculate the test performance of the bootstrap model in the original sample
    test_cstat_model <- roc(outcome~pr_test,data=data)
    results[i,4] <- test_cstat_model$auc
    test_citl_model <- glm(outcome ~ offset(lp_test),family=binomial, data=data)
    results[i,5] <- summary(test_citl_model)$coefficients[1,1]
    test_cslope_model <- glm(outcome ~ lp_test,family=binomial, data=data)
    results[i,6] <- summary(test_cslope_model)$coefficients[2,1]
 
    print(i)
  }
  results2 <- as.data.frame(results)
  colnames(results2) <- c("app_c_stat","app_citl","app_c_slope","test_c_stat","test_citl","test_c_slope")
  return(results2)
}
boot_results <- manual_boot(smart,50)


The optimisum adjusted statistics can then be obtained.  
We can also calculate the optimism in the validation statistics from validating in the development data.

In [ ]:
# optimism adjusted statistics
c1$auc - (mean(boot_results$app_c_stat)-mean(boot_results$test_c_stat)) # c-stat
mod_log_1$coef - (mean(boot_results$app_citl)-mean(boot_results$test_citl)) # citl
mod_log_2$coef[2]- (mean(boot_results$app_c_slope)-mean(boot_results$test_c_slope)) # c-slope

#optimism on its own
mean(boot_results$app_c_stat)-mean(boot_results$test_c_stat)
mean(boot_results$app_citl)-mean(boot_results$test_citl)
mean(boot_results$app_c_slope)-mean(boot_results$test_c_slope)

### Can you interpret the results? What do you conclude about the model performance?

We have also provided you a CHD data set that is discussed in the intro to R file.

Now you have seen how to build a model why not try and develop a logistic model for CHD.
Note, when loading in the excel file, some variable are not imported as numeric. See "converting a factor to numeric" section in the intro to R. 